<a href="https://colab.research.google.com/github/mervecakir1/Satellite-Segmentation/blob/main/huawei.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch, torchvision
print("Torch:", torch.__version__, "| TorchVision:", torchvision.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))


Torch: 2.8.0+cu126 | TorchVision: 0.23.0+cu126
CUDA available: True
GPU: NVIDIA L4


In [1]:
!pip install -U albumentations==1.4.8 albucore==0.0.12 opencv-python-headless==4.10.0.84 --no-cache-dir

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 177.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.8/156.8 kB 319.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 120.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 107.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.3/242.3 kB 338.9 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.12.0.88
    Uninstalling opencv-python-headless-4.12.0.88:
      Successfully uninstalled opencv-python-headless-4.12.0.88
  Attempting uninstall: albucore
    Found existing installation: albucore 0.0.24
    Uninstalling albucore-0.0.24:
      Successfully uninstalled albucore-0.0.24
  Attempting uninstall: albumentations
    Found existing 

In [ ]:
!pip install -q segmentation-models-pytorch==0.3.3
!pip install -q torchmetrics==1.3.2


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.5/68.5 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 841.5/841.5 kB 21.4 MB/s eta 0:00:00


In [ ]:
import kagglehub

path = kagglehub.dataset_download("mohammedjaveed/loveda-dataset")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/loveda-dataset


In [ ]:
import os, glob, shutil, random
from pathlib import Path


SRC = {
    "Train": "/kaggle/input/loveda-dataset/Train/Train/Urban",
    "Val"  : "/kaggle/input/loveda-dataset/Val/Val/Urban",
    "Test" : "/kaggle/input/loveda-dataset/Test/Test/Urban",
}
DST_ROOT = "/kaggle/working/LoveDA_Urban_subset"


SEED = 42
random.seed(SEED)


LIMITS = {
    "Train": 480,
    "Val"  : 220,
    "Test" : 320,
}

RATIOS = {
    "Train": 0.3,
    "Val"  : 0.3,
    "Test" : 0.3,
}

def ensure_dir(p): Path(p).mkdir(parents=True, exist_ok=True)

def pick_subset(items, split):
    if LIMITS.get(split) is not None:
        n = min(LIMITS[split], len(items))
    elif RATIOS.get(split) is not None:
        n = max(1, int(len(items) * float(RATIOS[split])))
    else:
        n = len(items)
    return random.sample(items, n) if n < len(items) else list(items)

def copy_split(split):
    urban_root = SRC[split]
    src_img = os.path.join(urban_root, "images_png")
    src_msk = os.path.join(urban_root, "masks_png")

    dst_img = os.path.join(DST_ROOT, split, "Image")
    dst_msk = os.path.join(DST_ROOT, split, "Mask")
    ensure_dir(dst_img); ensure_dir(dst_msk)


    img_files = sorted(glob.glob(os.path.join(src_img, "*.png")))
    if not img_files:
        raise FileNotFoundError(f"Görsel bulunamadı: {src_img}")


    if os.path.isdir(src_msk):
        msk_files = sorted(glob.glob(os.path.join(src_msk, "*.png")))
        img_bases = {os.path.basename(p) for p in img_files}
        msk_bases = {os.path.basename(p) for p in msk_files}
        common = sorted(img_bases & msk_bases)
        if not common:
            raise RuntimeError(f"Eşleşen image–mask yok: {src_img} vs {src_msk}")

        chosen = pick_subset(common, split)
        for b in chosen:
            shutil.copy(os.path.join(src_img, b), os.path.join(dst_img, b))
            shutil.copy(os.path.join(src_msk, b), os.path.join(dst_msk, b))
        print(f" {split}: {len(chosen)} çift kopyalandı → {dst_img} & {dst_msk}")

    else:

        chosen = pick_subset(img_files, split)
        for p in chosen:
            b = os.path.basename(p)
            shutil.copy(p, os.path.join(dst_img, b))
        print(f" {split}: {len(chosen)} görüntü kopyalandı → {dst_img} (maske yok)")

for sp in ["Train", "Val", "Test"]:
    copy_split(sp)


def count_png(p): return len(glob.glob(os.path.join(p, "*.png")))
for sp in ["Train", "Val", "Test"]:
    ip = os.path.join(DST_ROOT, sp, "Image")
    mp = os.path.join(DST_ROOT, sp, "Mask")
    print(f"{sp}: Image={count_png(ip)} | Mask={count_png(mp) if os.path.isdir(mp) else 0}")


✅ Train: 480 çift kopyalandı → /kaggle/working/LoveDA_Urban_subset/Train/Image & /kaggle/working/LoveDA_Urban_subset/Train/Mask
✅ Val: 220 çift kopyalandı → /kaggle/working/LoveDA_Urban_subset/Val/Image & /kaggle/working/LoveDA_Urban_subset/Val/Mask
✅ Test: 320 görüntü kopyalandı → /kaggle/working/LoveDA_Urban_subset/Test/Image (maske yok)
Train: Image=480 | Mask=480
Val: Image=220 | Mask=220
Test: Image=320 | Mask=0


In [ ]:
import os, glob
from pathlib import Path
import numpy as np
from PIL import Image
from tqdm import tqdm

SRC_SUBSET = "/kaggle/working/LoveDA_Urban_subset"

DST_PATCH  = "/kaggle/working/LoveDA_Urban_256_subset"

PATCH = 256
SPLITS = ["Train", "Val", "Test"]

def ensure_dir(p): Path(p).mkdir(parents=True, exist_ok=True)

def tile_and_save(img_np, out_dir_img, base, with_mask=False, msk_np=None, out_dir_msk=None):
    h, w = img_np.shape[:2]
    assert h % PATCH == 0 and w % PATCH == 0, f"Boyutlar {h}x{w} PATCH={PATCH} ile tam bölünmüyor."
    rows, cols = h // PATCH, w // PATCH
    for r in range(rows):
        for c in range(cols):
            sy, sx = r*PATCH, c*PATCH
            img_patch = img_np[sy:sy+PATCH, sx:sx+PATCH]
            Image.fromarray(img_patch).save(os.path.join(out_dir_img, f"{base}_r{r}_c{c}.png"))
            if with_mask and out_dir_msk is not None:
                msk_patch = msk_np[sy:sy+PATCH, sx:sx+PATCH]
                Image.fromarray(msk_patch.astype(np.uint8)).save(
                    os.path.join(out_dir_msk, f"{base}_r{r}_c{c}.png")
                )

for sp in SPLITS:
    src_img_dir = os.path.join(SRC_SUBSET, sp, "Image")
    src_msk_dir = os.path.join(SRC_SUBSET, sp, "Mask")

    out_img_dir = os.path.join(DST_PATCH, sp, "Image")
    out_msk_dir = os.path.join(DST_PATCH, sp, "Mask")
    ensure_dir(out_img_dir)

    has_masks = os.path.isdir(src_msk_dir)
    if has_masks:
        ensure_dir(out_msk_dir)

    img_files = sorted(glob.glob(os.path.join(src_img_dir, "*.png")))
    assert img_files, f"Görsel bulunamadı: {src_img_dir}"

    print(f"\n[{sp}] {len(img_files)} görüntü patch'leniyor...")
    for ip in tqdm(img_files):
        base = os.path.splitext(os.path.basename(ip))[0]
        img = np.array(Image.open(ip).convert("RGB"))
        h, w = img.shape[:2]

        if h % PATCH != 0 or w % PATCH != 0:
            new_h = ((h // PATCH) + 1) * PATCH if h % PATCH != 0 else h
            new_w = ((w // PATCH) + 1) * PATCH if w % PATCH != 0 else w

            padded_img = np.zeros((new_h, new_w, 3), dtype=img.dtype)
            padded_img[:h, :w] = img
            img = padded_img
            print(f"Görüntü {base} boyutu {h}x{w} -> {new_h}x{new_w} padding uygulandı")

        if has_masks:
            mp = os.path.join(src_msk_dir, base + ".png")
            if os.path.exists(mp):
                msk = np.array(Image.open(mp))

                if msk.shape[0] != img.shape[0] or msk.shape[1] != img.shape[1]:
                    padded_msk = np.zeros((img.shape[0], img.shape[1]), dtype=msk.dtype)
                    padded_msk[:msk.shape[0], :msk.shape[1]] = msk
                    msk = padded_msk

                tile_and_save(img, out_img_dir, base, with_mask=True, msk_np=msk, out_dir_msk=out_msk_dir)
            else:
                print(f"Mask bulunamadı: {mp}, sadece görüntü patch'leniyor")
                tile_and_save(img, out_img_dir, base, with_mask=False)
        else:
            tile_and_save(img, out_img_dir, base, with_mask=False)

print("\n Patch çıkarma tamam. Çıktı kökü:", DST_PATCH)


[Train] 480 görüntü patch'leniyor...


100%|██████████| 480/480 [04:26<00:00,  1.80it/s]



[Val] 220 görüntü patch'leniyor...


100%|██████████| 220/220 [01:56<00:00,  1.89it/s]



[Test] 320 görüntü patch'leniyor...


...

Mask bulunamadı: /kaggle/working/LoveDA_Urban_subset/Test/Mask/5983.png, sadece görüntü patch'leniyor


100%|█████████▉| 319/320 [02:21<00:00,  2.62it/s]

Mask bulunamadı: /kaggle/working/LoveDA_Urban_subset/Test/Mask/5986.png, sadece görüntü patch'leniyor


100%|██████████| 320/320 [02:21<00:00,  2.26it/s]


✅ Patch çıkarma tamam. Çıktı kökü: /kaggle/working/LoveDA_Urban_256_subset


In [ ]:
import os, glob

DST_PATCH = "/kaggle/working/LoveDA_Urban_256_subset"
for sp in ["Train","Val","Test"]:
    n_img = len(glob.glob(os.path.join(DST_PATCH, sp, "Image", "*.png")))
    n_msk = len(glob.glob(os.path.join(DST_PATCH, sp, "Mask",  "*.png")))
    print(f"{sp}: Image patches={n_img} | Mask patches={n_msk}")


Train: Image patches=7680 | Mask patches=7680
Val: Image patches=3520 | Mask patches=3520
Test: Image patches=5120 | Mask patches=0


In [ ]:
import os, glob, re, time, csv, json
import numpy as np
from PIL import Image

import torch
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2

WORK_DIR = "/kaggle/working" if os.path.exists("/kaggle/working") else "."
def wpath(name): return os.path.join(WORK_DIR, name)

DATA_ROOT = wpath("LoveDA_Urban_256_subset")
SPLITS = {
    "train": dict(img=f"{DATA_ROOT}/Train/Image", msk=f"{DATA_ROOT}/Train/Mask"),
    "val":   dict(img=f"{DATA_ROOT}/Val/Image",   msk=f"{DATA_ROOT}/Val/Mask"),
    "test":  dict(img=f"{DATA_ROOT}/Test/Image",  msk=f"{DATA_ROOT}/Test/Mask")
}

NUM_CLASSES  = 7
IGNORE_INDEX = 255
BATCH_SIZE   = 8
NUM_WORKERS  = 2
PIN_MEMORY   = True
IMG_SIZE     = 256

MEAN = (0.485, 0.456, 0.406)
STD  = (0.229, 0.224, 0.225)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


Device: cuda


In [ ]:
train_tfms = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.RandomRotate90(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.03, scale_limit=0.05, rotate_limit=10,
                       border_mode=cv2.BORDER_CONSTANT, value=(0,0,0), mask_value=0, p=0.3),
    A.RandomBrightnessContrast(p=0.3),
    A.Normalize(mean=MEAN, std=STD),
    ToTensorV2()
])
val_tfms = A.Compose([
    A.Normalize(mean=MEAN, std=STD),
    ToTensorV2()
])

# Dataset
class SegPatchDataset(Dataset):
    def __init__(self, img_dir, msk_dir=None, transform=None, ignore_index=255):
        self.imgs = sorted(glob.glob(os.path.join(img_dir, "*.png")))
        self.msk_dir = msk_dir if (msk_dir and os.path.isdir(msk_dir) and glob.glob(os.path.join(msk_dir, "*.png"))) else None
        self.transform = transform
        self.ignore_index = ignore_index
        if not self.imgs: raise RuntimeError(f"Boş klasör: {img_dir}")

        # LUT: 0->255, 1..7->0..6
        self.lut = np.full(256, 255, np.uint8)
        self.lut[1]=0; self.lut[2]=1; self.lut[3]=2; self.lut[4]=3; self.lut[5]=4; self.lut[6]=5; self.lut[7]=6

    def __len__(self): return len(self.imgs)

    def __getitem__(self, i):
        name = os.path.basename(self.imgs[i])
        img  = np.array(Image.open(self.imgs[i]).convert("RGB"))

        if self.msk_dir:
            mp = os.path.join(self.msk_dir, name)
            if os.path.isfile(mp):
                msk_raw = np.array(Image.open(mp), dtype=np.uint8)
                msk = self.lut[msk_raw]
            else:
                msk = np.full(img.shape[:2], 0, np.uint8)
        else:
            msk = np.full(img.shape[:2], 0, np.uint8)

        if self.transform:
            out = self.transform(image=img, mask=msk)
            img, msk = out["image"], out["mask"]
        else:
            img = torch.from_numpy(img).permute(2,0,1).float()/255.0
            msk = torch.from_numpy(msk)

        return img, msk.long(), name

def make_loader(split):
    cfg = SPLITS[split]
    use_msk = (split!="test") and bool(glob.glob(os.path.join(cfg["msk"], "*.png")))
    ds = SegPatchDataset(cfg["img"], cfg["msk"] if use_msk else None,
                         transform=(train_tfms if split=="train" else val_tfms),
                         ignore_index=IGNORE_INDEX)
    dl = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=(split=="train"),
                    num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY, drop_last=(split=="train"))
    return ds, dl

train_ds, train_loader = make_loader("train")
val_ds,   val_loader   = make_loader("val")
test_ds,  test_loader  = make_loader("test")
print("Train samples:", len(train_ds), "Val samples:", len(val_ds), "Test samples:", len(test_ds))
xb, yb, nb = next(iter(train_loader)); print("Batch:", xb.shape, yb.shape, nb[:3])


Train samples: 7680 Val samples: 3520 Test samples: 5120
Batch: torch.Size([8, 3, 256, 256]) torch.Size([8, 256, 256]) ['2222_r1_c3.png', '1525_r2_c2.png', '1619_r2_c3.png']


In [ ]:
def compute_class_weights(mask_dir, num_classes=NUM_CLASSES, clip_min=0.5, clip_max=5.0):
    lut = np.full(256, 255, np.uint8)
    lut[1]=0; lut[2]=1; lut[3]=2; lut[4]=3; lut[5]=4; lut[6]=5; lut[7]=6
    counts = np.zeros(num_classes, dtype=np.int64)
    mask_files = sorted(glob.glob(os.path.join(mask_dir, "*.png")))
    if not mask_files: return None
    for mp in mask_files:
        m = np.array(Image.open(mp), dtype=np.uint8)
        m = lut[m]
        for c in range(num_classes): counts[c] += (m == c).sum()
    freq = counts / max(1, counts.sum())
    med  = np.median(freq[freq > 0]) if np.any(freq>0) else 1.0
    w = (med / np.clip(freq, 1e-12, None)).astype(np.float32)
    w = np.clip(w, clip_min, clip_max)
    return torch.tensor(w, dtype=torch.float32)

CLASS_WEIGHTS = compute_class_weights(SPLITS["train"]["msk"])
if CLASS_WEIGHTS is not None: CLASS_WEIGHTS = CLASS_WEIGHTS.to(device)
print("Class Weights:", None if CLASS_WEIGHTS is None else CLASS_WEIGHTS.detach().cpu().numpy().round(3).tolist())


Class Weights: [0.5, 0.5, 0.8090000152587891, 2.428999900817871, 1.0850000381469727, 1.0, 5.0]


In [ ]:
import matplotlib.pyplot as plt
from tqdm import tqdm
from torch.optim import AdamW
from torch.cuda.amp import autocast, GradScaler
from torch.optim.lr_scheduler import OneCycleLR
import segmentation_models_pytorch as smp

@torch.no_grad()
def confusion_matrix(pred, target, C, ignore_index=IGNORE_INDEX):
    valid = (target != ignore_index)
    t = target[valid].view(-1)
    p = pred[valid].view(-1)
    cm = torch.bincount(t*C + p, minlength=C*C).reshape(C, C).to(torch.long)
    return cm

def metrics_from_cm(cm):
    cm = cm.float()
    TP = torch.diag(cm); FP = cm.sum(0) - TP; FN = cm.sum(1) - TP
    iou_c  = TP / (TP + FP + FN).clamp(min=1)
    dice_c = (2*TP) / (2*TP + FP + FN).clamp(min=1)
    acc    = TP.sum() / cm.sum().clamp(min=1)
    prec_c = TP / (TP + FP).clamp(min=1)
    rec_c  = TP / (TP + FN).clamp(min=1)
    f1_c   = 2*prec_c*rec_c / (prec_c + rec_c).clamp(min=1e-12)
    return {
        "mean_iou": iou_c.mean().item(),
        "mean_dice": dice_c.mean().item(),
        "accuracy": acc.item(),
        "mean_precision": prec_c.mean().item(),
        "mean_recall": rec_c.mean().item(),
        "mean_f1": f1_c.mean().item(),
        "per_class_iou": iou_c.detach().cpu().numpy(),
        "per_class_dice": dice_c.detach().cpu().numpy()
    }

# Loss
def dice_loss_mc(logits, target, ignore_index=IGNORE_INDEX, eps=1e-6):
    C = logits.shape[1]
    valid = (target != ignore_index)
    if valid.sum() == 0: return torch.tensor(0., device=logits.device)
    t = target.clone(); t[~valid] = 0
    oh = torch.zeros((logits.size(0), C, logits.size(2), logits.size(3)),
                     device=logits.device, dtype=torch.float32)
    oh.scatter_(1, t.unsqueeze(1), 1.0)
    oh = oh * valid.unsqueeze(1)
    prob = torch.softmax(logits, dim=1) * valid.unsqueeze(1)
    inter = (prob * oh).sum(dim=(0,2,3))
    den   = prob.sum(dim=(0,2,3)) + oh.sum(dim=(0,2,3)) + eps
    return 1. - (2.*inter / den).mean()

def combined_loss(logits, target, class_weights=None):
    ce = F.cross_entropy(logits, target, weight=class_weights, ignore_index=IGNORE_INDEX)
    dl = dice_loss_mc(logits, target, ignore_index=IGNORE_INDEX)
    return 0.7*ce + 0.3*dl

# Model
model = smp.Unet(encoder_name="resnet50", encoder_weights="imagenet",
                 classes=NUM_CLASSES, activation=None).to(device)

EPOCHS   = 50
LR       = 8e-4
WD       = 1e-4
PATIENCE = 10

optimizer = AdamW(model.parameters(), lr=LR, weight_decay=WD)
scheduler = OneCycleLR(optimizer, max_lr=LR, epochs=EPOCHS, steps_per_epoch=len(train_loader),
                       pct_start=0.1, div_factor=10.0, final_div_factor=100.0)
scaler = GradScaler()

BEST_CKPT  = wpath("best_unet_50epochs.pth")
CSV_HISTORY= wpath("unet_training_history.csv")
PLOT_PNG   = wpath("unet_training_results.png")

history = {"epoch":[], "train_loss":[], "val_loss":[], "mIoU":[], "Acc":[], "Dice":[], "F1":[]}
best = {"epoch":0, "mIoU":-1.0, "metrics":None}
with open(CSV_HISTORY, "w", newline="") as f:
    csv.writer(f).writerow(["epoch","train_loss","val_loss","mIoU","Acc","Dice","F1"])

epochs_no_improve = 0
for epoch in range(1, EPOCHS+1):
    # TRAIN
    model.train(); total_loss = 0.0
    for xb, yb, _ in tqdm(train_loader, desc=f"Train {epoch}/{EPOCHS}", leave=False):
        xb, yb = xb.to(device, non_blocking=True), yb.to(device, non_blocking=True)
        optimizer.zero_grad(set_to_none=True)
        with autocast():
            logits = model(xb)
            loss = combined_loss(logits, yb, class_weights=CLASS_WEIGHTS)
        scaler.scale(loss).backward()
        scaler.step(optimizer); scaler.update()
        scheduler.step()
        total_loss += loss.item() * xb.size(0)
    train_loss = total_loss / max(1, len(train_loader.dataset))

    # VAL
    model.eval(); cm_total = torch.zeros(NUM_CLASSES, NUM_CLASSES, dtype=torch.long, device=device); val_loss=0.0
    with torch.no_grad():
        for xb, yb, _ in tqdm(val_loader, desc="Val", leave=False):
            xb, yb = xb.to(device), yb.to(device)
            logits = model(xb)
            val_loss += combined_loss(logits, yb, class_weights=CLASS_WEIGHTS).item() * xb.size(0)
            preds = logits.argmax(1)
            cm_total += confusion_matrix(preds, yb, C=NUM_CLASSES, ignore_index=IGNORE_INDEX)
    val_loss /= max(1, len(val_loader.dataset))
    mets = metrics_from_cm(cm_total)

    # LOG
    history["epoch"].append(epoch)
    for k,v in [("train_loss",train_loss),("val_loss",val_loss),
                ("mIoU",mets["mean_iou"]),("Acc",mets["accuracy"]),
                ("Dice",mets["mean_dice"]),("F1",mets["mean_f1"])]:
        history[k].append(v)
    with open(CSV_HISTORY, "a", newline="") as f:
        csv.writer(f).writerow([epoch, f"{train_loss:.6f}", f"{val_loss:.6f}",
                                f"{mets['mean_iou']:.6f}", f"{mets['accuracy']:.6f}",
                                f"{mets['mean_dice']:.6f}", f"{mets['mean_f1']:.6f}"])

    print(f"[{epoch:03d}] Train:{train_loss:.4f} | Val:{val_loss:.4f} | "
          f"mIoU:{mets['mean_iou']:.3f} Dice:{mets['mean_dice']:.3f} Acc:{mets['accuracy']:.3f} F1:{mets['mean_f1']:.3f}")

    # BEST SAVE (mIoU) + Early Stop
    if mets["mean_iou"] > best["mIoU"]:
        best.update({"epoch":epoch, "mIoU":mets["mean_iou"], "metrics":mets})
        torch.save({
            "model_state_dict": model.state_dict(),
            "epoch": epoch,
            "metrics": mets,
            "history": history,
            "config": {"num_classes": NUM_CLASSES, "ignore_index": IGNORE_INDEX,
                       "img_size": IMG_SIZE, "mean": MEAN, "std": STD}
        }, BEST_CKPT)
        print(f" Best updated: mIoU={mets['mean_iou']:.4f} → {BEST_CKPT}")
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= PATIENCE:
            print(f"Early stopping @ {epoch}. Best mIoU={best['mIoU']:.4f} (epoch {best['epoch']})")
            break

plt.figure(figsize=(10,4))
plt.plot(history["epoch"], history["train_loss"], label="Train Loss")
plt.plot(history["epoch"], history["val_loss"],   label="Val Loss")
plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.legend(); plt.title("UNet Training Loss")
plt.tight_layout(); plt.savefig(PLOT_PNG); plt.close()

print("Eğitim tamam. Best epoch:", best["epoch"], "mIoU:", f"{best['mIoU']:.4f}")
print("Saved:", BEST_CKPT, "\nLogs:", CSV_HISTORY, "\nPlots:", PLOT_PNG)


Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 197MB/s]
/tmp/ipython-input-2880790886.py:69: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


🚀 Eğitim başlıyor...


Train 1/50:   0%|          | 0/960 [00:00<?, ?it/s]/tmp/ipython-input-2880790886.py:88: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


[001] Train:1.1639 | Val:1.3273 | mIoU:0.389 Dice:0.551 Acc:0.563 F1:0.551
  ✅ Best updated: mIoU=0.3894 → /kaggle/working/best_unet_50epochs.pth


...

[024] Train:0.6829 | Val:1.3749 | mIoU:0.433 Dice:0.594 Acc:0.589 F1:0.594


[025] Train:0.6810 | Val:1.6302 | mIoU:0.415 Dice:0.570 Acc:0.565 F1:0.570


[026] Train:0.6783 | Val:1.6777 | mIoU:0.409 Dice:0.559 Acc:0.544 F1:0.559


[027] Train:0.6570 | Val:1.3990 | mIoU:0.406 Dice:0.562 Acc:0.576 F1:0.562


[028] Train:0.6542 | Val:1.6660 | mIoU:0.402 Dice:0.563 Acc:0.564 F1:0.563
⏹️ Early stopping @ 28. Best mIoU=0.4439 (epoch 18)
✅ Eğitim tamam. Best epoch: 18 mIoU: 0.4439
Saved: /kaggle/working/best_unet_50epochs.pth 
Logs: /kaggle/working/unet_training_history.csv 
Plots: /kaggle/working/unet_training_results.png


In [ ]:
import csv, numpy as np, torch
from tqdm import tqdm
import segmentation_models_pytorch as smp

CKPT_PATH = wpath("best_unet_50epochs.pth")

def load_ckpt_safe(path):
    try:
        from torch.serialization import add_safe_globals
        import numpy as np
        add_safe_globals([np.core.multiarray._reconstruct])
        ckpt = torch.load(path, map_location="cpu", weights_only=True)
        if isinstance(ckpt, dict) and ("model_state_dict" in ckpt or "state_dict" in ckpt):
            state = ckpt.get("model_state_dict", ckpt.get("state_dict"))
            return state, ckpt

        if isinstance(ckpt, dict):
            return ckpt, {"_raw": "state_dict_only"}
    except Exception as e:
        print(f"[info] Safe load (weights_only=True) olmadı: {e}")

    ckpt = torch.load(path, map_location="cpu", weights_only=False)
    if isinstance(ckpt, dict) and ("model_state_dict" in ckpt or "state_dict" in ckpt):
        state = ckpt.get("model_state_dict", ckpt.get("state_dict"))
        return state, ckpt
    return ckpt, {"_raw": "state_dict_only"}

state_dict, meta = load_ckpt_safe(CKPT_PATH)

model_eval = smp.Unet(encoder_name="resnet50", encoder_weights=None,
                      classes=NUM_CLASSES, activation=None).to(device).eval()
model_eval.load_state_dict(state_dict)

@torch.no_grad()
def confusion_matrix(pred, target, C, ignore_index=IGNORE_INDEX):
    valid = (target != ignore_index)
    t = target[valid].view(-1); p = pred[valid].view(-1)
    return torch.bincount(t*C + p, minlength=C*C).reshape(C, C).to(torch.long)

def metrics_from_cm(cm):
    cm = cm.float()
    TP = torch.diag(cm); FP = cm.sum(0) - TP; FN = cm.sum(1) - TP
    iou_c  = TP / (TP + FP + FN).clamp(min=1)
    dice_c = (2*TP) / (2*TP + FP + FN).clamp(min=1)
    acc    = TP.sum() / cm.sum().clamp(min=1)
    prec_c = TP / (TP + FP).clamp(min=1)
    rec_c  = TP / (TP + FN).clamp(min=1)
    f1_c   = 2*prec_c*rec_c / (prec_c + rec_c).clamp(min=1e-12)
    return {
        "mean_iou": iou_c.mean().item(),
        "mean_dice": dice_c.mean().item(),
        "accuracy": acc.item(),
        "mean_precision": prec_c.mean().item(),
        "mean_recall": rec_c.mean().item(),
        "mean_f1": f1_c.mean().item(),
        "per_class_iou": iou_c.detach().cpu().numpy(),
        "per_class_dice": dice_c.detach().cpu().numpy()
    }

cm_total = torch.zeros(NUM_CLASSES, NUM_CLASSES, dtype=torch.long, device=device)
with torch.no_grad():
    for xb, yb, _ in tqdm(val_loader, desc="Eval/CSV"):
        xb, yb = xb.to(device), yb.to(device)
        preds = model_eval(xb).argmax(1)
        cm_total += confusion_matrix(preds, yb, C=NUM_CLASSES, ignore_index=IGNORE_INDEX)

mets = metrics_from_cm(cm_total)
print("VAL overall:",
      {k: round(v,4) for k,v in mets.items() if isinstance(v, float)})

CSV_OVERALL = wpath("unet_50epochs_metrics_overall.csv")
CSV_PERCLS  = wpath("unet_50epochs_metrics_per_class.csv")
CLASS_NAMES = ['Background','Building','Road','Water','Barren','Forest','Agriculture']

with open(CSV_OVERALL, "w", newline="") as f:
    w = csv.writer(f); w.writerow(["metric","value"])
    for k in ["mean_iou","mean_dice","accuracy","mean_precision","mean_recall","mean_f1"]:
        w.writerow([k, f"{mets[k]:.6f}"])

with open(CSV_PERCLS, "w", newline="") as f:
    w = csv.writer(f); w.writerow(["class_id","class_name","IoU","Dice"])
    for i, name in enumerate(CLASS_NAMES):
        w.writerow([i, name, f"{mets['per_class_iou'][i]:.6f}", f"{mets['per_class_dice'][i]:.6f}"])

print("Saved CSVs:", CSV_OVERALL, CSV_PERCLS)


[info] Safe load (weights_only=True) olmadı: Weights only load failed. This file can still be loaded, to do so you have two options, do those steps only if you trust the source of the checkpoint. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL numpy.ndarray was not an allowed global by default. Please use `torch.serialization.add_safe_globals([numpy.ndarray])` or the `torch.serialization.safe_globals([numpy.ndarray])` context manager to allowlist this global if you trust this class/function.

Check the documentation of torch.load to learn more about types accepted by

Eval/CSV: 100%|██████████| 440/440 [00:09<00:00, 44.83it/s]

VAL overall: {'mean_iou': 0.4439, 'mean_dice': 0.6062, 'accuracy': 0.6049, 'mean_precision': 0.6069, 'mean_recall': 0.6365, 'mean_f1': 0.6062}
Saved CSVs: /kaggle/working/unet_50epochs_metrics_overall.csv /kaggle/working/unet_50epochs_metrics_per_class.csv


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

COLOR_MAP = {
    0: (255, 255, 255),  # Background (beyaz)
    1: (255,   0,   0),  # Building (kırmızı)
    2: (255, 255,   0),  # Road (sarı)
    3: (  0,   0, 255),  # Water (mavi)
    4: (159, 129, 183),  # Barren (mor)
    5: (  0, 255,   0),  # Forest (yeşil)
    6: (255, 195, 128)   # Agricultural (turuncu)
}
PALETTE = np.array([COLOR_MAP[i] for i in range(0, 7)], dtype=np.uint8)

def colorize(mask_hw: np.ndarray) -> np.ndarray:
    rgb = np.zeros((mask_hw.shape[0], mask_hw.shape[1], 3), dtype=np.uint8)
    valid = (mask_hw >= 0) & (mask_hw < len(PALETTE))
    rgb[valid] = PALETTE[mask_hw[valid]]
    return rgb

@torch.no_grad()
def predict_with_tta(x, model_for_infer):
    logits = model_for_infer(x)
    logits_flip = model_for_infer(torch.flip(x, dims=[-1]))
    logits_flip = torch.flip(logits_flip, dims=[-1])
    return 0.5 * (logits + logits_flip)

SAVE_VAL  = wpath("unet_preds_val");  os.makedirs(SAVE_VAL,  exist_ok=True)
SAVE_TEST = wpath("unet_preds_test"); os.makedirs(SAVE_TEST, exist_ok=True)

@torch.no_grad()
def predict_and_save(dl, save_dir, desc):
    for xb, _, names in tqdm(dl, desc=desc):
        xb = xb.to(device, non_blocking=True)
        logits = predict_with_tta(xb, model_eval)
        preds  = logits.argmax(1).cpu().numpy().astype(np.uint8)
        for p, name in zip(preds, names):
            Image.fromarray(colorize(p)).save(os.path.join(save_dir, name))

predict_and_save(val_loader,  SAVE_VAL,  "VAL predict→PNG (B palette, TTA)")
predict_and_save(test_loader, SAVE_TEST, "TEST predict→PNG (B palette, TTA)")
print("VAL preds:", SAVE_VAL, "| TEST preds:", SAVE_TEST)

TILE = IMG_SIZE
VAL_IMG_DIR = SPLITS["val"]["img"]
VAL_MSK_DIR = SPLITS["val"]["msk"]
PRED_DIR    = SAVE_VAL

rx = re.compile(r"^(?P<base>.+)_r(?P<r>\d+)_c(?P<c>\d+)\.png$")
LUT = np.full(256, 255, np.uint8); LUT[1]=0; LUT[2]=1; LUT[3]=2; LUT[4]=3; LUT[5]=4; LUT[6]=5; LUT[7]=6

def draw_grid(img, tile=TILE, lw=1):
    out = img.copy()
    for y in range(tile, img.shape[0], tile): out[y-lw:y+lw, :] = 255
    for x in range(tile, img.shape[1], tile): out[:, x-lw:x+lw] = 255
    return out

def find_base_ids(img_dir):
    files = sorted(glob.glob(os.path.join(img_dir, "*.png")))
    bases = {}
    for fp in files:
        m = rx.match(os.path.basename(fp))
        if not m: continue
        b = m.group("base"); r = int(m.group("r")); c = int(m.group("c"))
        R, C, cnt = bases.get(b, (0,0,0))
        bases[b] = (max(R, r+1), max(C, c+1), cnt+1)
    return [b for b,_ in sorted(bases.items(), key=lambda kv: -kv[1][2])]

def reconstruct_full(img_dir, base):
    pats = sorted(glob.glob(os.path.join(img_dir, f"{base}_r*_c*.png")))
    assert pats, f"Patch yok: {base}"
    rs, cs = [], []
    for p in pats:
        m = rx.match(os.path.basename(p)); rs.append(int(m.group("r"))); cs.append(int(m.group("c")))
    R, C = max(rs)+1, max(cs)+1
    H, W = R*TILE, C*TILE
    full_img = np.zeros((H, W, 3), dtype=np.uint8)
    for p in pats:
        m = rx.match(os.path.basename(p)); r, c = int(m.group("r")), int(m.group("c"))
        img = np.array(Image.open(p).convert("RGB"))
        full_img[r*TILE:(r+1)*TILE, c*TILE:(c+1)*TILE] = img
    return full_img

base_ids = find_base_ids(VAL_IMG_DIR)
assert base_ids, "Val/Image içinde patch bulunamadı."
BASE = base_ids[1]

full_img      = reconstruct_full(VAL_IMG_DIR, BASE)
full_img_grid = draw_grid(full_img, tile=TILE, lw=1)

pred_patch_files = sorted(glob.glob(os.path.join(PRED_DIR, f"{BASE}_r*_c*.png")))
rs, cs = [], []
for p in pred_patch_files:
    m = rx.match(os.path.basename(p)); rs.append(int(m.group("r"))); cs.append(int(m.group("c")))
R, C = max(rs)+1, max(cs)+1
H, W = R*TILE, C*TILE
full_pred_rgb = np.zeros((H, W, 3), dtype=np.uint8)
for p in pred_patch_files:
    m = rx.match(os.path.basename(p)); r, c = int(m.group("r")), int(m.group("c"))
    rgb = np.array(Image.open(p).convert("RGB"))
    full_pred_rgb[r*TILE:(r+1)*TILE, c*TILE:(c+1)*TILE] = rgb

full_gt_rgb = None
if VAL_MSK_DIR and os.path.isdir(VAL_MSK_DIR):
    gt_patch_files = sorted(glob.glob(os.path.join(VAL_MSK_DIR, f"{BASE}_r*_c*.png")))
    if gt_patch_files:
        rs, cs = [], []
        for p in gt_patch_files:
            m = rx.match(os.path.basename(p)); rs.append(int(m.group("r"))); cs.append(int(m.group("c")))
        Rg, Cg = max(rs)+1, max(cs)+1
        Hg, Wg = Rg*TILE, Cg*TILE
        full_gt = np.zeros((Hg, Wg), dtype=np.uint8)

        for p in gt_patch_files:
            m = rx.match(os.path.basename(p)); r, c = int(m.group("r")), int(m.group("c"))
            msk_raw = np.array(Image.open(p), dtype=np.uint8)
            if msk_raw.max() > 6:
                msk = LUT[msk_raw]
            else:
                msk = msk_raw
            full_gt[r*TILE:(r+1)*TILE, c*TILE:(c+1)*TILE] = msk

        full_gt_rgb = colorize(full_gt)

if full_gt_rgb is not None:
    plt.figure(figsize=(20,5))
    plt.subplot(1,4,1); plt.imshow(full_img);      plt.title(f"Image (base={BASE})"); plt.axis('off')
    plt.subplot(1,4,2); plt.imshow(full_img_grid); plt.title("Patch Grid");          plt.axis('off')
    plt.subplot(1,4,3); plt.imshow(full_gt_rgb);   plt.title("Ground Truth");        plt.axis('off')
    plt.subplot(1,4,4); plt.imshow(full_pred_rgb); plt.title("U-Net Prediction");    plt.axis('off')
    plt.tight_layout()
    OUT_FULL = wpath(f"vis_full_grid_pred_{BASE}_withGT.png")
else:
    plt.figure(figsize=(16,5))
    plt.subplot(1,3,1); plt.imshow(full_img);      plt.title(f"Image (base={BASE})"); plt.axis('off')
    plt.subplot(1,3,2); plt.imshow(full_img_grid); plt.title("Patch Grid");          plt.axis('off')
    plt.subplot(1,3,3); plt.imshow(full_pred_rgb); plt.title("U-Net Prediction");    plt.axis('off')
    plt.tight_layout()
    OUT_FULL = wpath(f"vis_full_grid_pred_{BASE}.png")

plt.savefig(OUT_FULL, dpi=150); plt.close()
print(" Kaydedildi:", OUT_FULL)


TEST predict→PNG (B palette, TTA): 100%|██████████| 640/640 [00:53<00:00, 11.94it/s]


VAL preds: /kaggle/working/unet_preds_val | TEST preds: /kaggle/working/unet_preds_test
✅ Kaydedildi: /kaggle/working/vis_full_grid_pred_3515_withGT.png


In [ ]:
#DeepLabV3+
import torch, csv
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import AdamW
from torch.cuda.amp import autocast, GradScaler
from torch.optim.lr_scheduler import OneCycleLR
import matplotlib.pyplot as plt
from tqdm import tqdm
import segmentation_models_pytorch as smp

#Metrikler
@torch.no_grad()
def confusion_matrix(pred, target, C, ignore_index=IGNORE_INDEX):
    valid = (target != ignore_index)
    t = target[valid].view(-1); p = pred[valid].view(-1)
    return torch.bincount(t*C + p, minlength=C*C).reshape(C, C).to(torch.long)

def metrics_from_cm(cm):
    cm = cm.float()
    TP = torch.diag(cm); FP = cm.sum(0) - TP; FN = cm.sum(1) - TP
    iou_c  = TP / (TP + FP + FN).clamp(min=1)
    dice_c = (2*TP) / (2*TP + FP + FN).clamp(min=1)
    acc    = TP.sum() / cm.sum().clamp(min=1)
    prec_c = TP / (TP + FP).clamp(min=1)
    rec_c  = TP / (TP + FN).clamp(min=1)
    f1_c   = 2*prec_c*rec_c / (prec_c + rec_c).clamp(min=1e-12)
    return {
        "mean_iou": iou_c.mean().item(),
        "mean_dice": dice_c.mean().item(),
        "accuracy": acc.item(),
        "mean_precision": prec_c.mean().item(),
        "mean_recall": rec_c.mean().item(),
        "mean_f1": f1_c.mean().item(),
        "per_class_iou": iou_c.detach().cpu().numpy(),
        "per_class_dice": dice_c.detach().cpu().numpy()
    }

#Loss
def dice_loss_mc(logits, target, ignore_index=IGNORE_INDEX, eps=1e-6):
    C = logits.shape[1]
    valid = (target != ignore_index)
    if valid.sum() == 0: return torch.tensor(0., device=logits.device)
    t = target.clone(); t[~valid] = 0
    oh = torch.zeros((logits.size(0), C, logits.size(2), logits.size(3)),
                     device=logits.device, dtype=torch.float32)
    oh.scatter_(1, t.unsqueeze(1), 1.0)
    oh = oh * valid.unsqueeze(1)
    prob = torch.softmax(logits, dim=1) * valid.unsqueeze(1)
    inter = (prob * oh).sum(dim=(0,2,3))
    den   = prob.sum(dim=(0,2,3)) + oh.sum(dim=(0,2,3)) + eps
    return 1. - (2.*inter / den).mean()

def combined_loss(logits, target, class_weights=None):
    ce = F.cross_entropy(logits, target, weight=class_weights, ignore_index=IGNORE_INDEX)
    dl = dice_loss_mc(logits, target, ignore_index=IGNORE_INDEX)
    return 0.7*ce + 0.3*dl

# Model
model = smp.DeepLabV3Plus(
    encoder_name="resnet50", encoder_weights="imagenet",
    classes=NUM_CLASSES, activation=None
).to(device)


EPOCHS   = 50
LR       = 8e-4
WD       = 1e-4
PATIENCE = 10

optimizer = AdamW(model.parameters(), lr=LR, weight_decay=WD)
scheduler = OneCycleLR(optimizer, max_lr=LR, epochs=EPOCHS, steps_per_epoch=len(train_loader),
                       pct_start=0.1, div_factor=10.0, final_div_factor=100.0)
scaler = GradScaler()

BEST_CKPT  = wpath("best_deeplabv3plus_50epochs.pth")
CSV_HISTORY= wpath("deeplabv3plus_training_history.csv")
PLOT_PNG   = wpath("deeplabv3plus_training_results.png")


history = {"epoch":[], "train_loss":[], "val_loss":[], "mIoU":[], "Acc":[], "Dice":[], "F1":[]}
best = {"epoch":0, "mIoU":-1.0, "metrics":None}
with open(CSV_HISTORY, "w", newline="") as f:
    csv.writer(f).writerow(["epoch","train_loss","val_loss","mIoU","Acc","Dice","F1"])

epochs_no_improve = 0
for epoch in range(1, EPOCHS+1):
    # TRAIN
    model.train(); total_loss = 0.0
    for xb, yb, _ in tqdm(train_loader, desc=f"Train {epoch}/{EPOCHS}", leave=False):
        xb, yb = xb.to(device, non_blocking=True), yb.to(device, non_blocking=True)
        optimizer.zero_grad(set_to_none=True)
        with autocast():
            logits = model(xb)
            loss = combined_loss(logits, yb, class_weights=CLASS_WEIGHTS)
        scaler.scale(loss).backward()
        scaler.step(optimizer); scaler.update()
        scheduler.step()
        total_loss += loss.item() * xb.size(0)
    train_loss = total_loss / max(1, len(train_loader.dataset))

    # VAL
    model.eval(); cm_total = torch.zeros(NUM_CLASSES, NUM_CLASSES, dtype=torch.long, device=device); val_loss=0.0
    with torch.no_grad():
        for xb, yb, _ in tqdm(val_loader, desc="Val", leave=False):
            xb, yb = xb.to(device), yb.to(device)
            logits = model(xb)
            val_loss += combined_loss(logits, yb, class_weights=CLASS_WEIGHTS).item() * xb.size(0)
            preds = logits.argmax(1)
            cm_total += confusion_matrix(preds, yb, C=NUM_CLASSES, ignore_index=IGNORE_INDEX)
    val_loss /= max(1, len(val_loader.dataset))
    mets = metrics_from_cm(cm_total)

    # LOG
    history["epoch"].append(epoch)
    for k,v in [("train_loss",train_loss),("val_loss",val_loss),
                ("mIoU",mets["mean_iou"]),("Acc",mets["accuracy"]),
                ("Dice",mets["mean_dice"]),("F1",mets["mean_f1"])]:
        history[k].append(v)
    with open(CSV_HISTORY, "a", newline="") as f:
        csv.writer(f).writerow([epoch, f"{train_loss:.6f}", f"{val_loss:.6f}",
                                f"{mets['mean_iou']:.6f}", f"{mets['accuracy']:.6f}",
                                f"{mets['mean_dice']:.6f}", f"{mets['mean_f1']:.6f}"])

    print(f"[{epoch:03d}] Train:{train_loss:.4f} | Val:{val_loss:.4f} | "
          f"mIoU:{mets['mean_iou']:.3f} Dice:{mets['mean_dice']:.3f} Acc:{mets['accuracy']:.3f} F1:{mets['mean_f1']:.3f}")

    # BEST SAVE (mIoU) + Early Stop
    if mets["mean_iou"] > best["mIoU"]:
        best.update({"epoch":epoch, "mIoU":mets["mean_iou"], "metrics":mets})
        torch.save({
            "model_state_dict": model.state_dict(),
            "epoch": epoch,
            "metrics": mets,
            "history": history,
            "config": {"num_classes": NUM_CLASSES, "ignore_index": IGNORE_INDEX,
                       "img_size": IMG_SIZE, "mean": MEAN, "std": STD}
        }, BEST_CKPT)
        print(f"Best updated: mIoU={mets['mean_iou']:.4f} → {BEST_CKPT}")
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= PATIENCE:
            print(f"Early stopping @ {epoch}. Best mIoU={best['mIoU']:.4f} (epoch {best['epoch']})")
            break

plt.figure(figsize=(10,4))
plt.plot(history["epoch"], history["train_loss"], label="Train Loss")
plt.plot(history["epoch"], history["val_loss"],   label="Val Loss")
plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.legend(); plt.title("DeepLabV3+ Training Loss")
plt.tight_layout(); plt.savefig(PLOT_PNG); plt.close()

print(" Eğitim tamam. Best epoch:", best["epoch"], "mIoU:", f"{best['mIoU']:.4f}")
print("Saved:", BEST_CKPT, "\nLogs:", CSV_HISTORY, "\nPlots:", PLOT_PNG)


/tmp/ipython-input-1258812556.py:74: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


🚀 DeepLabV3+ Eğitim başlıyor...


Train 1/50:   0%|          | 0/960 [00:00<?, ?it/s]/tmp/ipython-input-1258812556.py:94: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


[001] Train:1.0169 | Val:1.3252 | mIoU:0.407 Dice:0.565 Acc:0.564 F1:0.565
  ✅ Best updated: mIoU=0.4067 → /kaggle/working/best_deeplabv3plus_50epochs.pth


[002] Train:0.9137 | Val:1.5127 | mIoU:0.344 Dice:0.489 Acc:0.521 F1:0.489


...

[013] Train:0.7746 | Val:1.2352 | mIoU:0.406 Dice:0.568 Acc:0.562 F1:0.568


[014] Train:0.7630 | Val:1.2368 | mIoU:0.401 Dice:0.553 Acc:0.575 F1:0.553


[015] Train:0.7443 | Val:1.3734 | mIoU:0.393 Dice:0.553 Acc:0.543 F1:0.553


[016] Train:0.7356 | Val:1.5718 | mIoU:0.342 Dice:0.502 Acc:0.530 F1:0.502


[017] Train:0.7068 | Val:1.3710 | mIoU:0.400 Dice:0.563 Acc:0.556 F1:0.563
⏹️ Early stopping @ 17. Best mIoU=0.4197 (epoch 7)
✅ Eğitim tamam. Best epoch: 7 mIoU: 0.4197
Saved: /kaggle/working/best_deeplabv3plus_50epochs.pth 
Logs: /kaggle/working/deeplabv3plus_training_history.csv 
Plots: /kaggle/working/deeplabv3plus_training_results.png


In [ ]:
from tqdm import tqdm
import csv

def evaluate(model, loader, device):
    model.eval()
    cm_total = torch.zeros(NUM_CLASSES, NUM_CLASSES, dtype=torch.long, device=device)

    with torch.no_grad():
        for xb, yb, _ in tqdm(loader, desc="Validation"):
            xb, yb = xb.to(device), yb.to(device)
            logits = model(xb)
            preds = logits.argmax(1)
            cm_total += confusion_matrix(preds, yb, C=NUM_CLASSES, ignore_index=IGNORE_INDEX)

    return metrics_from_cm(cm_total)

mets = evaluate(model, val_loader, device)

print(f" Overall Accuracy: {mets['accuracy']:.4f}")
print(f" Mean IoU: {mets['mean_iou']:.4f}")
print(f" Mean Dice: {mets['mean_dice']:.4f}")
print(f" Mean Precision: {mets['mean_precision']:.4f}")
print(f" Mean Recall: {mets['mean_recall']:.4f}")
print(f" Mean F1-Score: {mets['mean_f1']:.4f}")

for i, name in enumerate(CLASS_NAMES):
    print(f"{name:>11} {mets['per_class_iou'][i]:.3f} {mets['per_class_dice'][i]:.3f} "
          f"{mets['per_class_precision'][i]:.3f} {mets['per_class_recall'][i]:.3f} "
          f"{mets['per_class_f1'][i]:.3f}")

CSV_OVERALL = wpath("deeplabv3plus_metrics_overall.csv")
CSV_PERCLS  = wpath("deeplabv3plus_metrics_per_class.csv")

with open(CSV_OVERALL, "w", newline="") as f:
    w = csv.writer(f); w.writerow(["metric","value"])
    for k in ["mean_iou","mean_dice","accuracy","mean_precision","mean_recall","mean_f1"]:
        w.writerow([k, f"{mets[k]:.6f}"])

with open(CSV_PERCLS, "w", newline="") as f:
    w = csv.writer(f); w.writerow(["class_id","class_name","IoU","Dice","Precision","Recall","F1"])
    for i, name in enumerate(CLASS_NAMES):
        w.writerow([
            i, name,
            f"{mets['per_class_iou'][i]:.6f}",
            f"{mets['per_class_dice'][i]:.6f}",
            f"{mets['per_class_precision'][i]:.6f}",
            f"{mets['per_class_recall'][i]:.6f}",
            f"{mets['per_class_f1'][i]:.6f}"
        ])

print("\n CSV dosyaları kaydedildi:", CSV_OVERALL, "ve", CSV_PERCLS)


Validation: 100%|██████████| 440/440 [00:09<00:00, 46.13it/s]


📊 DEEPLABV3+ PERFORMANS ÖZETİ
🎯 Overall Accuracy: 6.1122
🎯 Mean IoU: 0.4002
🎯 Mean Dice: 0.5634
🎯 Mean Precision: 0.5984
🎯 Mean Recall: 0.5851
🎯 Mean F1-Score: 0.5634

📋 SINIF BAZINDA METRİKLER:
      Class   IoU  Dice Precision Recall    F1
 Background 0.354 0.523 0.524 0.523 0.523
   Building 0.490 0.657 0.606 0.719 0.657
       Road 0.503 0.669 0.717 0.627 0.669
      Water 0.553 0.712 0.807 0.637 0.712
     Barren 0.262 0.415 0.515 0.348 0.415
     Forest 0.356 0.525 0.370 0.907 0.525
Agriculture 0.283 0.442 0.651 0.334 0.442

✅ CSV dosyaları kaydedildi: /kaggle/working/deeplabv3plus_metrics_overall.csv ve /kaggle/working/deeplabv3plus_metrics_per_class.csv


In [ ]:
import os, glob, re, csv, time
import numpy as np
import torch
from tqdm import tqdm
import segmentation_models_pytorch as smp
from PIL import Image
import matplotlib.pyplot as plt

def find_deeplab_ckpt():
    candidates = []
    for base in [wpath(""), "/kaggle/working", "."]:
        try:
            for p in glob.glob(os.path.join(base, "*.pth")):
                name = os.path.basename(p).lower()
                if "deeplab" in name or "v3" in name:
                    candidates.append(p)
        except Exception:
            pass
    if not candidates:
        raise FileNotFoundError("DeepLabV3+ checkpoint bulunamadı.")
    candidates.sort(key=lambda p: os.path.getmtime(p), reverse=True)
    return candidates[0]

CKPT_PATH = wpath("best_deeplabv3plus_50epochs.pth")
if not os.path.isfile(CKPT_PATH):
    CKPT_PATH = find_deeplab_ckpt()
print(" Kullanılacak checkpoint:", CKPT_PATH)

def load_ckpt_safe(path):
    try:
        from torch.serialization import add_safe_globals
        import numpy as np
        add_safe_globals([np.core.multiarray._reconstruct])
        ckpt = torch.load(path, map_location="cpu", weights_only=True)
        if isinstance(ckpt, dict) and ("model_state_dict" in ckpt or "state_dict" in ckpt):
            return ckpt.get("model_state_dict", ckpt.get("state_dict")), ckpt
        if isinstance(ckpt, dict):
            return ckpt, {"_raw": "state_dict_only"}
    except Exception as e:
        print(f"[info] Safe load (weights_only=True) olmadı: {e}")
    ckpt = torch.load(path, map_location="cpu", weights_only=False)
    if isinstance(ckpt, dict) and ("model_state_dict" in ckpt or "state_dict" in ckpt):
        return ckpt.get("model_state_dict", ckpt.get("state_dict")), ckpt
    return ckpt, {"_raw": "state_dict_only"}

state_dict, meta = load_ckpt_safe(CKPT_PATH)

model_eval = smp.DeepLabV3Plus(
    encoder_name="resnet50",
    encoder_weights=None,
    classes=NUM_CLASSES,
    activation=None
).to(device).eval()
model_eval.load_state_dict(state_dict)

COLOR_MAP = {
    0: (255, 255, 255),  # Background
    1: (255,   0,   0),  # Building
    2: (255, 255,   0),  # Road
    3: (  0,   0, 255),  # Water
    4: (159, 129, 183),  # Barren
    5: (  0, 255,   0),  # Forest
    6: (255, 195, 128)   # Agricultural
}
PALETTE = np.array([COLOR_MAP[i] for i in range(0, 7)], dtype=np.uint8)

def colorize(mask_hw: np.ndarray) -> np.ndarray:
    rgb = np.zeros((mask_hw.shape[0], mask_hw.shape[1], 3), dtype=np.uint8)
    valid = (mask_hw >= 0) & (mask_hw < len(PALETTE))
    rgb[valid] = PALETTE[mask_hw[valid]]
    return rgb

SAVE_VAL  = wpath("deeplab_preds_val");  os.makedirs(SAVE_VAL,  exist_ok=True)

TILE = IMG_SIZE
VAL_IMG_DIR = SPLITS["val"]["img"]
PRED_DIR    = SAVE_VAL

rx = re.compile(r"^(?P<base>.+)_r(?P<r>\d+)_c(?P<c>\d+)\.png$")

def draw_grid(img, tile=TILE, lw=1):
    out = img.copy()
    for y in range(tile, img.shape[0], tile): out[y-lw:y+lw, :] = 255
    for x in range(tile, img.shape[1], tile): out[:, x-lw:x+lw] = 255
    return out

def find_base_ids(img_dir):
    files = sorted(glob.glob(os.path.join(img_dir, "*.png")))
    bases = {}
    for fp in files:
        m = rx.match(os.path.basename(fp))
        if not m: continue
        b = m.group("base"); r = int(m.group("r")); c = int(m.group("c"))
        R, C, cnt = bases.get(b, (0,0,0))
        bases[b] = (max(R, r+1), max(C, c+1), cnt+1)
    return [b for b,_ in sorted(bases.items(), key=lambda kv: -kv[1][2])]

def reconstruct_full(img_dir, base):
    pats = sorted(glob.glob(os.path.join(img_dir, f"{base}_r*_c*.png")))
    assert pats, f"Patch yok: {base}"
    rs, cs = [], []
    for p in pats:
        m = rx.match(os.path.basename(p)); rs.append(int(m.group("r"))); cs.append(int(m.group("c")))
    R, C = max(rs)+1, max(cs)+1
    H, W = R*TILE, C*TILE
    full_img = np.zeros((H, W, 3), dtype=np.uint8)
    for p in pats:
        m = rx.match(os.path.basename(p)); r, c = int(m.group("r")), int(m.group("c"))
        img = np.array(Image.open(p).convert("RGB"))
        full_img[r*TILE:(r+1)*TILE, c*TILE:(c+1)*TILE] = img
    return full_img

BASE = "3515"
base_ids = find_base_ids(VAL_IMG_DIR)
if BASE not in base_ids and len(base_ids)>0:
    BASE = base_ids[0]
print("Seçilen base:", BASE)

full_img      = reconstruct_full(VAL_IMG_DIR, BASE)
full_img_grid = draw_grid(full_img, tile=TILE, lw=1)


pred_patch_files = sorted(glob.glob(os.path.join(PRED_DIR, f"{BASE}_r*_c*.png")))
rs, cs = [], []
for p in pred_patch_files:
    m = rx.match(os.path.basename(p)); rs.append(int(m.group("r"))); cs.append(int(m.group("c")))
R, C = max(rs)+1, max(cs)+1
H, W = R*TILE, C*TILE
full_pred_rgb = np.zeros((H, W, 3), dtype=np.uint8)
for p in pred_patch_files:
    m = rx.match(os.path.basename(p)); r, c = int(m.group("r")), int(m.group("c"))
    rgb = np.array(Image.open(p).convert("RGB"))
    full_pred_rgb[r*TILE:(r+1)*TILE, c*TILE:(c+1)*TILE] = rgb

plt.figure(figsize=(16,5))
plt.subplot(1,3,1); plt.imshow(full_img);      plt.title(f"Image (base={BASE})"); plt.axis('off')
plt.subplot(1,3,2); plt.imshow(full_img_grid); plt.title("Patch Grid");           plt.axis('off')
plt.subplot(1,3,3); plt.imshow(full_pred_rgb); plt.title("DeepLabV3+ Prediction"); plt.axis('off')
plt.tight_layout()
OUT_FULL = wpath(f"vis_full_grid_pred_{BASE}_deeplab.png")

plt.savefig(OUT_FULL, dpi=150); plt.close()
print("Kaydedildi:", OUT_FULL)


✅ Kullanılacak checkpoint: /kaggle/working/best_deeplabv3plus_50epochs.pth
[info] Safe load (weights_only=True) olmadı: Weights only load failed. This file can still be loaded, to do so you have two options, do those steps only if you trust the source of the checkpoint. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL numpy.ndarray was not an allowed global by default. Please use `torch.serialization.add_safe_globals([numpy.ndarray])` or the `torch.serialization.safe_globals([numpy.ndarray])` context manager to allowlist this global if you trust this class/function.



In [ ]:
# FCN
import os, time, csv, glob, re
import numpy as np
import torch
import torch.nn.functional as F
from torch.optim import AdamW
from torch.cuda.amp import autocast, GradScaler
from torch.optim.lr_scheduler import OneCycleLR
from tqdm import tqdm
import matplotlib.pyplot as plt
from PIL import Image

if 'wpath' not in globals():
    def wpath(name): return os.path.join("/kaggle/working", name)

if 'CLASS_WEIGHTS' not in globals() or CLASS_WEIGHTS is None:
    CLASS_WEIGHTS = torch.ones(NUM_CLASSES, dtype=torch.float32, device=device)

def dice_loss_mc(logits, target, ignore_index=255, eps=1e-6):
    C = logits.shape[1]
    valid = (target != ignore_index)
    if valid.sum() == 0:
        return torch.tensor(0., device=logits.device)

    t = target.clone(); t[~valid] = 0
    pred = F.softmax(logits, dim=1)
    pred = pred.permute(0,2,3,1).reshape(-1, C)
    tgt1h = F.one_hot(t.reshape(-1), C).float().to(logits.device)
    tgt1h[~valid.reshape(-1)] = 0

    num = (pred * tgt1h).sum(0) * 2
    den = pred.sum(0) + tgt1h.sum(0) + eps
    dice = 1 - (num / den).mean()
    return dice

# loss (CE + Dice)

def combined_loss(logits, target, ce_w=0.5, dice_w=0.5):
    ce = F.cross_entropy(logits, target, weight=CLASS_WEIGHTS, ignore_index=IGNORE_INDEX)
    dice = dice_loss_mc(logits, target, ignore_index=IGNORE_INDEX)
    return ce_w*ce + dice_w*dice

from torchvision.models.segmentation import fcn_resnet50
model = fcn_resnet50(num_classes=NUM_CLASSES).to(device)
scaler = GradScaler()


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 182MB/s]
/tmp/ipython-input-1642511436.py:58: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()


In [ ]:
# FCN Train
import csv, torch
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch.nn.functional as F

BEST_CKPT   = wpath("best_fcn_resnet50_50epochs.pth")
CSV_HISTORY = wpath("fcn_training_history.csv")
PLOT_PNG    = wpath("fcn_training_results.png")

@torch.no_grad()
def confusion_matrix(pred, target, C, ignore_index=IGNORE_INDEX):
    valid = (target != ignore_index)
    t = target[valid].view(-1)
    p = pred[valid].view(-1)
    return torch.bincount(t*C + p, minlength=C*C).reshape(C, C).to(torch.long)

def metrics_from_cm(cm):
    cm = cm.float()
    TP = torch.diag(cm); FP = cm.sum(0) - TP; FN = cm.sum(1) - TP
    iou_c  = TP / (TP + FP + FN).clamp(min=1)
    dice_c = (2*TP) / (2*TP + FP + FN).clamp(min=1)
    acc    = TP.sum() / cm.sum().clamp(min=1)
    prec_c = TP / (TP + FP).clamp(min=1)
    rec_c  = TP / (TP + FN).clamp(min=1)
    f1_c   = 2*prec_c*rec_c / (prec_c + rec_c).clamp(min=1e-12)
    return {
        "mean_iou": iou_c.mean().item(),
        "mean_dice": dice_c.mean().item(),
        "accuracy": acc.item(),
        "mean_precision": prec_c.mean().item(),
        "mean_recall": rec_c.mean().item(),
        "mean_f1": f1_c.mean().item()
    }

#Optimizasyon & LR planı
from torch.optim import AdamW
from torch.optim.lr_scheduler import OneCycleLR
optimizer = AdamW(model.parameters(), lr=8e-4, weight_decay=1e-4)

EPOCHS   = 50
PATIENCE = 10
scheduler = OneCycleLR(
    optimizer,
    max_lr=8e-4,
    steps_per_epoch=len(train_loader),
    epochs=EPOCHS,
    pct_start=0.1, div_factor=10.0, final_div_factor=100.0
)

with open(CSV_HISTORY, "w", newline="") as f:
    csv.writer(f).writerow(["epoch","train_loss","val_loss","mIoU","Acc","Dice","F1"])

history = {"epoch":[], "train_loss":[], "val_loss":[], "mIoU":[], "Acc":[], "Dice":[], "F1":[]}
best = {"epoch":0, "mIoU":-1.0, "metrics":None}

epochs_no_improve = 0
for epoch in range(1, EPOCHS+1):
    # TRAIN
    model.train(); total_loss = 0.0
    for xb, yb, _ in tqdm(train_loader, desc=f"Train {epoch}/{EPOCHS}", leave=False):
        xb, yb = xb.to(device, non_blocking=True), yb.to(device, non_blocking=True)
        optimizer.zero_grad(set_to_none=True)
        with autocast():
            out = model(xb)
            logits = out["out"] if isinstance(out, dict) else out
            loss = combined_loss(logits, yb)
        scaler.scale(loss).backward()
        scaler.step(optimizer); scaler.update()
        scheduler.step()
        total_loss += loss.item() * xb.size(0)
    train_loss = total_loss / max(1, len(train_loader.dataset))

    #VAL
    model.eval(); cm_total = torch.zeros(NUM_CLASSES, NUM_CLASSES, dtype=torch.long, device=device); val_loss=0.0
    with torch.no_grad():
        for xb, yb, _ in tqdm(val_loader, desc="Val", leave=False):
            xb, yb = xb.to(device), yb.to(device)
            out = model(xb)
            logits = out["out"] if isinstance(out, dict) else out
            val_loss += combined_loss(logits, yb).item() * xb.size(0)
            preds = logits.argmax(1)
            cm_total += confusion_matrix(preds, yb, C=NUM_CLASSES, ignore_index=IGNORE_INDEX)
    val_loss /= max(1, len(val_loader.dataset))
    mets = metrics_from_cm(cm_total)

    # LOG & CSV
    history["epoch"].append(epoch)
    for k,v in [("train_loss",train_loss),("val_loss",val_loss),
                ("mIoU",mets["mean_iou"]),("Acc",mets["accuracy"]),
                ("Dice",mets["mean_dice"]),("F1",mets["mean_f1"])]:
        history[k].append(v)
    with open(CSV_HISTORY, "a", newline="") as f:
        csv.writer(f).writerow([epoch, f"{train_loss:.6f}", f"{val_loss:.6f}",
                                f"{mets['mean_iou']:.6f}", f"{mets['accuracy']:.6f}",
                                f"{mets['mean_dice']:.6f}", f"{mets['mean_f1']:.6f}"])

    print(f"[{epoch:03d}] Train:{train_loss:.4f} | Val:{val_loss:.4f} | "
          f"mIoU:{mets['mean_iou']:.3f} Dice:{mets['mean_dice']:.3f} Acc:{mets['accuracy']:.3f} F1:{mets['mean_f1']:.3f}")

    # BEST SAVE (mIoU) + Early Stop
    if mets["mean_iou"] > best["mIoU"]:
        best.update({"epoch":epoch, "mIoU":mets["mean_iou"], "metrics":mets})
        torch.save({
            "model_state_dict": model.state_dict(),
            "epoch": epoch,
            "metrics": mets,
            "history": history,
            "config": {"num_classes": NUM_CLASSES, "ignore_index": IGNORE_INDEX}
        }, BEST_CKPT)
        print(f" Best updated: mIoU={mets['mean_iou']:.4f} → {BEST_CKPT}")
        epochs_no_improve = 0
    else:
        epochs_no_improve += 1
        if epochs_no_improve >= PATIENCE:
            print(f" Early stopping @ {epoch}. Best mIoU={best['mIoU']:.4f} (epoch {best['epoch']})")
            break

plt.figure(figsize=(10,4))
plt.plot(history["epoch"], history["train_loss"], label="Train Loss")
plt.plot(history["epoch"], history["val_loss"],   label="Val Loss")
plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.legend(); plt.title("FCN Training Loss")
plt.tight_layout(); plt.savefig(PLOT_PNG); plt.close()

print("FCN eğitim tamam. Best epoch:", best["epoch"], "mIoU:", f"{best['mIoU']:.4f}")
print("Saved:", BEST_CKPT, "\nLogs:", CSV_HISTORY, "\nPlots:", PLOT_PNG)


🚀 FCN eğitim başlıyor...


Train 1/50:   0%|          | 0/960 [00:00<?, ?it/s]/tmp/ipython-input-1934636952.py:68: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


[001] Train:0.8640 | Val:1.2644 | mIoU:0.382 Dice:0.530 Acc:0.544 F1:0.530
  ✅ Best updated: mIoU=0.3816 → /kaggle/working/best_fcn_resnet50_50epochs.pth


[002] Train:0.8231 | Val:1.2896 | mIoU:0.367 Dice:0.524 Acc:0.519 F1:0.524


[003] Train:0.8312 | Val:1.2503 | mIoU:0.380 Dice:0.537 Acc:0.537 F1:0.537


...

[020] Train:0.5847 | Val:1.2839 | mIoU:0.397 Dice:0.560 Acc:0.569 F1:0.560


[021] Train:0.5736 | Val:1.7687 | mIoU:0.384 Dice:0.525 Acc:0.529 F1:0.525


[022] Train:0.5696 | Val:1.2515 | mIoU:0.402 Dice:0.567 Acc:0.559 F1:0.567


[023] Train:0.5604 | Val:1.5936 | mIoU:0.395 Dice:0.546 Acc:0.552 F1:0.546


[024] Train:0.5404 | Val:1.7052 | mIoU:0.380 Dice:0.529 Acc:0.530 F1:0.529
⏹️ Early stopping @ 24. Best mIoU=0.4308 (epoch 14)
✅ FCN eğitim tamam. Best epoch: 14 mIoU: 0.4308
Saved: /kaggle/working/best_fcn_resnet50_50epochs.pth 
Logs: /kaggle/working/fcn_training_history.csv 
Plots: /kaggle/working/fcn_training_results.png


In [ ]:
import os, glob, re, csv, time
import numpy as np
import torch
from tqdm import tqdm
from PIL import Image
import matplotlib.pyplot as plt

def wpath(p):
    base = "/kaggle/working"
    return p if p.startswith(base) else os.path.join(base, p)

COLOR_MAP = {
    0:(255,255,255), 1:(255,0,0), 2:(255,255,0),
    3:(0,0,255), 4:(159,129,183), 5:(0,255,0), 6:(255,195,128)
}
PALETTE = np.array([COLOR_MAP[i] for i in range(0,7)], dtype=np.uint8)

def colorize(mask_hw: np.ndarray) -> np.ndarray:
    rgb = np.zeros((mask_hw.shape[0], mask_hw.shape[1], 3), dtype=np.uint8)
    valid = (mask_hw >= 0) & (mask_hw < len(PALETTE))
    rgb[valid] = PALETTE[mask_hw[valid]]
    return rgb

@torch.no_grad()
def confusion_matrix(pred, target, C, ignore_index=None):
    valid = (target != ignore_index) if ignore_index is not None else torch.ones_like(target, dtype=torch.bool)
    t = target[valid].view(-1)
    p = pred[valid].view(-1)
    return torch.bincount(t*C + p, minlength=C*C).reshape(C, C).to(torch.long)

def metrics_from_cm(cm: torch.Tensor):
    eps = 1e-6
    cm = cm.float()
    TP = torch.diag(cm)
    FP = cm.sum(0) - TP
    FN = cm.sum(1) - TP
    TN = cm.sum() - (TP + FP + FN)

    iou_c  = TP / (TP + FP + FN + eps)
    dice_c = 2*TP / (2*TP + FP + FN + eps)
    prec_c = TP / (TP + FP + eps)
    rec_c  = TP / (TP + FN + eps)
    f1_c   = 2*prec_c*rec_c / (prec_c + rec_c + eps)
    acc    = (TP + TN) / (TP + FP + FN + TN + eps)

    return {
        "accuracy": acc.mean().item(),
        "global_accuracy": (TP.sum()/cm.sum()).item(),
        "mean_iou": iou_c.mean().item(),
        "mean_dice": dice_c.mean().item(),
        "mean_precision": prec_c.mean().item(),
        "mean_recall": rec_c.mean().item(),
        "mean_f1": f1_c.mean().item(),
        "per_class_iou": iou_c.cpu().numpy(),
        "per_class_dice": dice_c.cpu().numpy(),
        "per_class_precision": prec_c.cpu().numpy(),
        "per_class_recall": rec_c.cpu().numpy(),
        "per_class_f1": f1_c.cpu().numpy(),
        "per_class_acc": acc.cpu().numpy(),
    }

import torchvision.models.segmentation as segm

CKPT_PATH = wpath("best_fcn_resnet50_50epochs.pth")
assert os.path.isfile(CKPT_PATH), f"Checkpoint yok: {CKPT_PATH}"

fcn_model = segm.fcn_resnet50(weights=None, num_classes=NUM_CLASSES).to(device).eval()

def load_ckpt_safe(path):
    try:
        from torch.serialization import add_safe_globals
        import numpy as np
        add_safe_globals([np.core.multiarray._reconstruct])
        state = torch.load(path, map_location="cpu", weights_only=True)
        if isinstance(state, dict) and "state_dict" in state:
            return state["state_dict"]
        return state
    except Exception as e:
        print(f"[info] weights_only=True olmadı: {e}")
        return torch.load(path, map_location="cpu", weights_only=False)

state_dict = load_ckpt_safe(CKPT_PATH)

if isinstance(state_dict, dict) and "model_state_dict" in state_dict:
    state_dict = state_dict["model_state_dict"]
fcn_model.load_state_dict(state_dict)

cm_total = torch.zeros(NUM_CLASSES, NUM_CLASSES, dtype=torch.long, device=device)
with torch.no_grad():
    for xb, yb, _ in tqdm(val_loader, desc="FCN Validation"):
        xb, yb = xb.to(device), yb.to(device)
        out = fcn_model(xb)
        logits = out["out"] if isinstance(out, dict) else out
        preds = logits.argmax(1)
        cm_total += confusion_matrix(preds, yb, C=NUM_CLASSES, ignore_index=IGNORE_INDEX)

mets = metrics_from_cm(cm_total)


print(f" Overall Pixel Accuracy: {mets['global_accuracy']:.4f}")
print(f"Mean (per-class) Accuracy: {mets['accuracy']:.4f}")
print(f"Mean IoU: {mets['mean_iou']:.4f}")
print(f" Mean Dice: {mets['mean_dice']:.4f}")
print(f" Mean Precision: {mets['mean_precision']:.4f}")
print(f"Mean Recall: {mets['mean_recall']:.4f}")
print(f" Mean F1-Score: {mets['mean_f1']:.4f}")

CLASS_NAMES = ['Background','Building','Road','Water','Barren','Forest','Agriculture']

for i, name in enumerate(CLASS_NAMES):
    print(f"{name:>11} "
          f"{mets['per_class_iou'][i]:.3f} {mets['per_class_dice'][i]:.3f} "
          f"{mets['per_class_acc'][i]:.3f} {mets['per_class_precision'][i]:.3f} "
          f"{mets['per_class_recall'][i]:.3f} {mets['per_class_f1'][i]:.3f}")

CSV_OVERALL = wpath("fcn_metrics_overall.csv")
CSV_PERCLS  = wpath("fcn_metrics_per_class.csv")
with open(CSV_OVERALL, "w", newline="") as f:
    w = csv.writer(f); w.writerow(["metric","value"])
    for k in ["global_accuracy","accuracy","mean_iou","mean_dice","mean_precision","mean_recall","mean_f1"]:
        w.writerow([k, f"{mets[k]:.6f}"])
with open(CSV_PERCLS, "w", newline="") as f:
    w = csv.writer(f); w.writerow(["class_id","class_name","IoU","Dice","Acc","Precision","Recall","F1"])
    for i, name in enumerate(CLASS_NAMES):
        w.writerow([
            i, name,
            f"{mets['per_class_iou'][i]:.6f}",
            f"{mets['per_class_dice'][i]:.6f}",
            f"{mets['per_class_acc'][i]:.6f}",
            f"{mets['per_class_precision'][i]:.6f}",
            f"{mets['per_class_recall'][i]:.6f}",
            f"{mets['per_class_f1'][i]:.6f}",
        ])
print("CSV'ler kaydedildi:", CSV_OVERALL, "ve", CSV_PERCLS)

@torch.no_grad()
def predict_and_save(dl, save_dir, desc):
    os.makedirs(save_dir, exist_ok=True)
    for xb, _, names in tqdm(dl, desc=desc):
        xb = xb.to(device, non_blocking=True)
        out = fcn_model(xb)
        logits = out["out"] if isinstance(out, dict) else out
        preds  = logits.argmax(1).cpu().numpy().astype(np.uint8)
        for p, name in zip(preds, names):
            Image.fromarray(colorize(p)).save(os.path.join(save_dir, name))

SAVE_VAL  = wpath("fcn_preds_val")
SAVE_TEST = wpath("fcn_preds_test")
predict_and_save(val_loader,  SAVE_VAL,  "VAL predict→PNG (FCN, B palette)")

print("VAL preds:", SAVE_VAL)

TILE = IMG_SIZE
VAL_IMG_DIR = SPLITS["val"]["img"]
PRED_DIR    = SAVE_VAL

rx = re.compile(r"^(?P<base>.+)_r(?P<r>\d+)_c(?P<c>\d+)\.png$")

def draw_grid(img, tile=TILE, lw=1):
    out = img.copy()
    for y in range(tile, img.shape[0], tile): out[y-lw:y+lw, :] = 255
    for x in range(tile, img.shape[1], tile): out[:, x-lw:x+lw] = 255
    return out

def find_base_ids(img_dir):
    files = sorted(glob.glob(os.path.join(img_dir, "*.png")))
    bases = {}
    for fp in files:
        m = rx.match(os.path.basename(fp))
        if not m: continue
        b = m.group("base"); r = int(m.group("r")); c = int(m.group("c"))
        R, C, cnt = bases.get(b, (0,0,0))
        bases[b] = (max(R, r+1), max(C, c+1), cnt+1)
    return [b for b,_ in sorted(bases.items(), key=lambda kv: -kv[1][2])]

def reconstruct_full(img_dir, base):
    pats = sorted(glob.glob(os.path.join(img_dir, f"{base}_r*_c*.png")))
    assert pats, f"Patch yok: {base}"
    rs, cs = [], []
    for p in pats:
        m = rx.match(os.path.basename(p)); rs.append(int(m.group("r"))); cs.append(int(m.group("c")))
    R, C = max(rs)+1, max(cs)+1
    H, W = R*TILE, C*TILE
    full_img = np.zeros((H, W, 3), dtype=np.uint8)
    for p in pats:
        m = rx.match(os.path.basename(p)); r, c = int(m.group("r")), int(m.group("c"))
        img = np.array(Image.open(p).convert("RGB"))
        full_img[r*TILE:(r+1)*TILE, c*TILE:(c+1)*TILE] = img
    return full_img

BASE = "3515"
base_ids = find_base_ids(VAL_IMG_DIR)
if BASE not in base_ids and len(base_ids)>0:
    BASE = base_ids[0]

full_img      = reconstruct_full(VAL_IMG_DIR, BASE)
full_img_grid = draw_grid(full_img, tile=TILE, lw=1)

pred_patch_files = sorted(glob.glob(os.path.join(PRED_DIR, f"{BASE}_r*_c*.png")))
rs, cs = [], []
for p in pred_patch_files:
    m = rx.match(os.path.basename(p)); rs.append(int(m.group("r"))); cs.append(int(m.group("c")))
R, C = max(rs)+1, max(cs)+1
H, W = R*TILE, C*TILE
full_pred_rgb = np.zeros((H, W, 3), dtype=np.uint8)
for p in pred_patch_files:
    m = rx.match(os.path.basename(p)); r, c = int(m.group("r")), int(m.group("c"))
    rgb = np.array(Image.open(p).convert("RGB"))
    full_pred_rgb[r*TILE:(r+1)*TILE, c*TILE:(c+1)*TILE] = rgb

plt.figure(figsize=(16,5))
plt.subplot(1,3,1); plt.imshow(full_img);      plt.title(f"Image (base={BASE})"); plt.axis('off')
plt.subplot(1,3,2); plt.imshow(full_img_grid); plt.title("Patch Grid");           plt.axis('off')
plt.subplot(1,3,3); plt.imshow(full_pred_rgb); plt.title("FCN Prediction");       plt.axis('off')
plt.tight_layout()
OUT_FULL = wpath(f"vis_full_grid_pred_{BASE}_fcn.png")
plt.savefig(OUT_FULL, dpi=150); plt.close()
print(" Kaydedildi:", OUT_FULL)

import pandas as pd
HIST_CSV = wpath("fcn_training_history.csv")
if os.path.isfile(HIST_CSV):
    dfh = pd.read_csv(HIST_CSV)
    plt.figure(figsize=(8,5))
    if "train_loss" in dfh.columns and "val_loss" in dfh.columns:
        plt.plot(dfh["train_loss"], label="Train Loss")
        plt.plot(dfh["val_loss"],   label="Val Loss")
    else:

        tr_col = "train_loss" if "train_loss" in dfh.columns else ("train" if "train" in dfh.columns else dfh.columns[0])
        va_col = "val_loss"   if "val_loss"   in dfh.columns else ("val"   if "val"   in dfh.columns else dfh.columns[1])
        plt.plot(dfh[tr_col], label="Train Loss")
        plt.plot(dfh[va_col], label="Val Loss")
    plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.title("FCN Training & Validation Loss")
    plt.legend(); plt.grid(True)
    OUT_LOSS = wpath("fcn_loss_curve.png")
    plt.savefig(OUT_LOSS, dpi=150); plt.close()
    print(f"Loss grafiği kaydedildi: {OUT_LOSS}")
else:
    print(f" Loss history CSV bulunamadı: {HIST_CSV}")


FCN Validation: 100%|██████████| 440/440 [00:16<00:00, 26.83it/s]



📊 FCN (ResNet50) PERFORMANS ÖZETİ
🎯 Overall Pixel Accuracy: 0.5725
🎯 Mean (per-class) Accuracy: 0.8779
🎯 Mean IoU: 0.4308
🎯 Mean Dice: 0.5849
🎯 Mean Precision: 0.6416
🎯 Mean Recall: 0.6208
🎯 Mean F1-Score: 0.5849

📋 SINIF BAZINDA METRİKLER:
      Class   IoU  Dice    Acc Precision Recall    F1
 Background 0.396 0.568 0.754 0.534 0.606 0.568
   Building 0.550 0.710 0.921 0.688 0.734 0.710
       Road 0.538 0.700 0.958 0.767 0.644 0.700
      Water 0.688 0.815 0.952 0.820 0.811 0.815
     Barren 0.328 0.494 0.961 0.536 0.458 0.494
     Forest 0.331 0.498 0.814 0.344 0.901 0.498
Agriculture 0.183 0.310 0.785 0.803 0.192 0.310
✅ CSV'ler kaydedildi: /kaggle/working/fcn_metrics_overall.csv ve /kaggle/working/fcn_metrics_per_class.csv


VAL predict→PNG (FCN, B palette): 100%|██████████| 440/440 [00:35<00:00, 12.48it/s]


📁 VAL preds: /kaggle/working/fcn_preds_val
🧩 Seçilen base: 3515
✅ Kaydedildi: /kaggle/working/vis_full_grid_pred_3515_fcn.png
📈 Loss grafiği kaydedildi: /kaggle/working/fcn_loss_curve.png


For Presentation


In [ ]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def _ensure_out(outdir):
    os.makedirs(outdir, exist_ok=True)

def _read_overall(path_overall):
    df = pd.read_csv(path_overall)
    if {"metric", "value"}.issubset(df.columns):
        return {row["metric"]: float(row["value"]) for _, row in df.iterrows()}
    return {k: float(v) for k, v in df.iloc[0].to_dict().items() if pd.api.types.is_number(v)}

def _read_perclass(path_perclass):
    df = pd.read_csv(path_perclass)
    rename_map = {}
    for c in df.columns:
        if c.lower() == "iou": rename_map[c] = "IoU"
        if c.lower() == "dice": rename_map[c] = "Dice"
        if c.lower() == "precision": rename_map[c] = "Precision"
        if c.lower() == "recall": rename_map[c] = "Recall"
        if c.lower() in ("f1","f1_score","f1-score"): rename_map[c] = "F1"
        if c.lower() in ("accuracy","acc"): rename_map[c] = "Accuracy"
        if c.lower() in ("class","classname","class_name"): rename_map[c] = "class_name"
    df = df.rename(columns=rename_map)
    if "class_name" not in df.columns:
        df["class_name"] = [f"Class {i}" for i in range(len(df))]
    for col in ["IoU","Dice","Precision","Recall","F1","Accuracy"]:
        if col in df.columns:
            df[col] = pd.to_numeric(df[col], errors="coerce")
    return df

def _plot_loss(history_csv, out_png, title="Training & Validation Loss"):
    if not os.path.isfile(history_csv):
        return
    df = pd.read_csv(history_csv)
    tr_col = None
    for k in ["train_loss","train","Train","training_loss"]:
        if k in df.columns: tr_col = k; break
    va_col = None
    for k in ["val_loss","val","Val","validation_loss"]:
        if k in df.columns: va_col = k; break
    if tr_col is None or va_col is None:
        return
    plt.figure(figsize=(8,5))
    plt.plot(df[tr_col].values, label="Train Loss")
    plt.plot(df[va_col].values, label="Val Loss")
    plt.xlabel("Epoch"); plt.ylabel("Loss"); plt.title(title)
    plt.grid(True, alpha=0.3); plt.legend()
    plt.tight_layout(); plt.savefig(out_png, dpi=150); plt.close()

def _bar(values, labels, ylabel, title, out_png, ylim01=True):
    plt.figure(figsize=(9,5))
    x = np.arange(len(labels))
    vals = np.array(values, dtype=float)
    plt.bar(x, vals)
    plt.xticks(x, labels, rotation=20)
    plt.ylabel(ylabel)
    plt.title(title)
    if ylim01:
        plt.ylim(0, 1.0)
    for i,v in enumerate(vals):
        plt.text(i, v + 0.01*(1 if ylim01 else max(vals, default=1)), f"{v:.3f}", ha="center", va="bottom", fontsize=9)
    plt.tight_layout(); plt.savefig(out_png, dpi=150); plt.close()

def generate_model_figures(
    model_name: str,
    overall_csv: str,
    perclass_csv: str,
    history_csv: str = None,
    outdir: str = "./figures"
):
    outdir = os.path.join(outdir, model_name.replace(" ", "_").lower())
    _ensure_out(outdir)

    overall = _read_overall(overall_csv)
    percls  = _read_perclass(perclass_csv)

    if history_csv:
        _plot_loss(history_csv, os.path.join(outdir, "loss_curve.png"), f"{model_name} — Training & Validation Loss")

    keys = []
    labels_map = {
        "global_accuracy":"Overall Acc",
        "accuracy":"Mean Acc",
        "mean_iou":"Mean IoU",
        "mean_dice":"Mean Dice",
        "mean_precision":"Mean Precision",
        "mean_recall":"Mean Recall",
        "mean_f1":"Mean F1",
    }
    for k in ["global_accuracy","accuracy","mean_iou","mean_dice","mean_precision","mean_recall","mean_f1"]:
        if k in overall: keys.append(k)

    if keys:
        _bar(
            [overall[k] for k in keys],
            [labels_map.get(k,k) for k in keys],
            ylabel="Score",
            title=f"{model_name} — Overall Metrics",
            out_png=os.path.join(outdir, "overall_bars.png"),
            ylim01=True
        )
        pd.DataFrame([overall]).to_csv(os.path.join(outdir, "overall_table.csv"), index=False)

    class_labels = percls["class_name"].tolist()

    metric_list = [("IoU","perclass_iou.png"),
                   ("Dice","perclass_dice.png"),
                   ("Precision","perclass_precision.png"),
                   ("Recall","perclass_recall.png"),
                   ("F1","perclass_f1.png")]

    if "Accuracy" in percls.columns:
        metric_list.append(("Accuracy","perclass_accuracy.png"))

    for met, filename in metric_list:
        if met in percls.columns:
            _bar(
                percls[met].values,
                class_labels,
                ylabel=met,
                title=f"{model_name} — Per-class {met}",
                out_png=os.path.join(outdir, filename),
                ylim01=True
            )

    print(f"{model_name}: çıktılar -> {outdir}")

# 1) U-Net
generate_model_figures(
    model_name   ="U-Net (ResNet50)",
    overall_csv  ="/kaggle/working/unet_50epochs_metrics_overall.csv",
    perclass_csv ="/kaggle/working/unet_50epochs_metrics_per_class.csv",
    history_csv  ="/kaggle/working/unet_training_history.csv",
    outdir       ="/kaggle/working/presentation_figs"
)

# 2) DeepLabV3+
generate_model_figures(
    model_name   ="DeepLabV3+ (ResNet50)",
    overall_csv  ="/kaggle/working/deeplabv3plus_metrics_overall.csv",
    perclass_csv ="/kaggle/working/deeplabv3plus_metrics_per_class.csv",
    history_csv  ="/kaggle/working/deeplab_training_history.csv",
    outdir       ="/kaggle/working/presentation_figs"
)

# 3) FCN
generate_model_figures(
    model_name   ="FCN (ResNet50)",
    overall_csv  ="/kaggle/working/fcn_metrics_overall.csv",
    perclass_csv ="/kaggle/working/fcn_metrics_per_class.csv",
    history_csv  ="/kaggle/working/fcn_training_history.csv",
    outdir       ="/kaggle/working/presentation_figs"
)


✅ U-Net (ResNet50): çıktılar -> /kaggle/working/presentation_figs/u-net_(resnet50)


/tmp/ipython-input-1474834811.py:73: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout(); plt.savefig(out_png, dpi=150); plt.close()


✅ DeepLabV3+ (ResNet50): çıktılar -> /kaggle/working/presentation_figs/deeplabv3+_(resnet50)


/tmp/ipython-input-1474834811.py:73: UserWarning: Tight layout not applied. The bottom and top margins cannot be made large enough to accommodate all Axes decorations.
  plt.tight_layout(); plt.savefig(out_png, dpi=150); plt.close()


✅ FCN (ResNet50): çıktılar -> /kaggle/working/presentation_figs/fcn_(resnet50)
